In [26]:
import requests
import csv
import pandas as pd
import os
import re

# Base URL for the ACS data
base_url = "https://api.census.gov/data/{year}/acs/acs5"

# Variables to fetch

#Houserent
variables = {'main_agg_rent': "NAME,B25060_001E", 'sub_median_rent': "NAME,B25058_001E", 
             'sub_lower_rent': "NAME,B25057_001E", 'sub_upper_rent': "NAME,B25059_001E"}

# Location filters
location = "for=tract:*&in=state:17+county:031"

In [30]:
def extract_info_from_filename(filename, prefix):
    if prefix == 'main':
        indicator_name = filename[5:-5]
        year = filename[-4:]
    else:
        indicator_name = filename[4:-5]
        year = filename[-4:]
    return indicator_name, year

all_dataframes = []

In [34]:
name = 'main_agg_rent_2015'
print(extract_info_from_filename(name, 'main'))

agg_rent
('agg_rent', '2015')


In [32]:
# Process each year from 2015 to 2019
for name, variable in variables.items():    

    ind_type = name.split('_')[0]
    print(ind_type)

    for year in range(2015, 2023):
        # Construct the URL for the current year
        url = f"{base_url.format(year=year)}?get={variable}&{location}"

        # Make the request
        response = requests.get(url)
        data = response.json()

        # Specify the path to save the CSV file, one for each year
        file_path = f'/home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Housing/ContractRent/{ind_type}/{name}_{year}.csv'
        
        # Open a CSV file for writing for each year
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)

            # Write the data into the CSV file
            writer.writerows(data)

        print(f"Data for {year} for {name} has been written to {file_path}")
    
    short_dir = f'/home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Housing/ContractRent/{ind_type}'
    
    if ind_type == 'main':    
        for filename in os.listdir(short_dir):
            if filename.endswith('.csv'):
                filepath = os.path.join(short_dir, filename)
            
                indicator_name, year = extract_info_from_filename(filename, ind_type)

                if indicator_name and year:
                    df = pd.read_csv(filepath)
                    # Create a new DataFrame with the required columns
                    new_df = pd.DataFrame({
                        'census_tract_id': df['tract'].astype(int),
                        'indicator_name': indicator_name,
                        'value': df.iloc[:, 1].fillna(0).astype(int),  # Assuming value is in the second column
                        'year': int(year)
                    })
                    all_dataframes.append(new_df)

        # Concatenate all DataFrames
        final_dataframe = pd.concat(all_dataframes)

        # Save to a new CSV file
        final_dataframe.to_csv('/path/to/save/aggregated_data.csv', index=False)

main
Data for 2015 for main_agg_rent has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Housing/ContractRent/main/main_agg_rent_2015.csv
Data for 2016 for main_agg_rent has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Housing/ContractRent/main/main_agg_rent_2016.csv
Data for 2017 for main_agg_rent has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Housing/ContractRent/main/main_agg_rent_2017.csv
Data for 2018 for main_agg_rent has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Housing/ContractRent/main/main_agg_rent_2018.csv
Data for 2019 for main_agg_rent has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Housing/ContractRent/main/main_agg_rent_2019.csv
Data for 2020 for main_agg_rent has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Housing/ContractRent/main/main

ValueError: invalid literal for int() with base 10: '.csv'